In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
import torch.optim as optim
from torch.jit import script, trace
from torch.utils.data import DataLoader, TensorDataset
import torchvision
from torchvision.transforms import v2

from tqdm import tqdm

from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score

In [9]:
x_train = None
y_train = None

x_valid = None
y_valid = None

x_test = None
y_test = None

In [47]:
train_subset = torch.utils.data.TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
val_subset = torch.utils.data.TensorDataset(torch.Tensor(x_valid), torch.Tensor(y_valid))
test_dataset = torch.utils.data.TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test))

bsz = 20
train_loader = torch.utils.data.DataLoader(train_subset, shuffle=True, batch_size=bsz)
val_loader = torch.utils.data.DataLoader(val_subset, shuffle=False, batch_size=bsz)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=bsz)

TypeError: new(): invalid data type 'str'

In [12]:
samp_time = 750
n_channels = 8

In [14]:
class CNN_LSTM(nn.Module):
    def __init__(self):
        super(CNN_LSTM, self).__init__()
        self.conv1d = nn.Conv1d(in_channels=8, out_channels=40, kernel_size=20, stride=4)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.batchnorm1 = nn.BatchNorm1d(40)
        self.maxpool = nn.MaxPool1d(kernel_size=4, stride=4)
        self.lstm1 = nn.LSTM(input_size=45, hidden_size=30)
        self.dropout2 = nn.Dropout(p=0.5)
        self.batchnorm2 = nn.BatchNorm1d(40)
        self.lstm2 = nn.LSTM(input_size=30, hidden_size=10)
        self.dropout3 = nn.Dropout(p=0.5)
        self.batchnorm3 = nn.BatchNorm1d(40)
        self.flatten = nn.Flatten()
        self.dense = nn.Linear(400,2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1d(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.batchnorm1(x)
        x = self.maxpool(x)
        x, _ = self.lstm1(x)
        x = self.dropout2(x)
        x = self.batchnorm2(x)
        x, _ = self.lstm2(x)
        x = self.dropout3(x)
        x = self.batchnorm3(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.softmax(x)

        return x

In [15]:
input_size = (n_channels, samp_time)

# Random input tensor with the specified dimensions
input_tensor = torch.randn(1, *input_size)
print(input_tensor.shape)

# Forward pass through the model
#output_tensor = model(input_tensor)

net = CNN_LSTM()
print(net.forward(Variable(input_tensor)))
criterion = nn.CrossEntropyLoss() # add to device here
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-7)

torch.Size([1, 8, 750])
tensor([[0.4091, 0.5909]], grad_fn=<SoftmaxBackward0>)


In [46]:
train_hist = []
val_hist = []

num_epochs = 100

for epoch_idx in tqdm(range(num_epochs)):
    net.train()
    train_count = 0
    train_correct_count = 0
    for batch_idx, (train_x, train_y) in enumerate(train_loader):
        train_x = train_x.float()
        train_y = train_y.long()
        optimizer.zero_grad()
        logits = net(train_x)
        loss = criterion(logits, train_y)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            y_hat = torch.argmax(logits, dim=-1)
            train_correct_count += torch.sum(y_hat == train_y, axis=-1)
            train_count += train_x.size(0)

    train_acc = train_correct_count / train_count
    train_hist.append(train_acc)

    net.eval()
    val_count = 0
    val_correct_count = 0
    with torch.no_grad():
        for idx, (val_x, val_y) in enumerate(val_loader):
            val_x = val_x.float()
            val_y = val_y.long()
            logits = net(val_x).detach()
            y_hat = torch.argmax(logits, dim=-1)
            val_correct_count += torch.sum(y_hat == val_y, axis=-1)
            val_count += val_x.size(0)
    val_acc = val_correct_count / val_count
    val_hist.append(val_acc)

    print('Train acc: {:.3f}, Val acc: {:.3f}'.format(train_acc, val_acc))

  0%|          | 0/100 [00:00<?, ?it/s]


NameError: name 'train_loader' is not defined

### Data

In [17]:
X = np.load('preprocessed_data.npy')

In [18]:
X = np.swapaxes(X, 1,2)

In [19]:
X.shape

(69, 8, 750)

In [20]:
x_train = X

In [22]:
y_train = 'valence'

### Labels

In [33]:
emocsv = pd.read_csv("emotion-data/Emotions! (Core) (Responses) - Form Responses 1.csv")
emocsv2 = emocsv.iloc[:,4:]

In [36]:
new_cols = ["How feel", "Pos", "Energ", "Dom", "Content", "Amused", "Angry", "Sad", "Disgust", "Afraid", "Emo"]

In [37]:
emo_labels = pd.DataFrame()

for row in range(3):
    sel_row = emocsv2.iloc[row]
    for i in np.arange(0, emocsv2.shape[1], 11):
        obs = sel_row[i:i+11].to_frame().T
        obs = obs.rename(columns={obs.columns[i]: new_cols[i] for i in range(11)})
        emo_labels = pd.concat([emo_labels, obs])

In [38]:
emo_labels['Pos2'] = (emo_labels['Pos'] > 4) + 0
emo_labels['Energ2'] = (emo_labels['Energ'] > 4) + 0
emo_labels['Dom2'] = (emo_labels['Dom'] > 4) + 0

In [39]:
valence = emo_labels['Pos2'].to_numpy()
valence

array([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1])

In [40]:
emo_labels

,How feel,Pos,Energ,Dom,Content,Amused,Angry,Sad,Disgust,Afraid,Emo,Pos2,Energ2,Dom2
0,"disgusted, but the music made the video funny,...",2,3,7,1,2,4,1,5,1,Disgusted,0,0,1
0,disgusted,2,3,7,1,1,1,1,5,1,Disgusted,0,0,1
0,disgusted,1,5,7,1,1,1,1,5,1,Disgusted,0,1,1
0,relaxed,6,5,8,3,1,1,1,1,1,Neutral (unemotional),1,1,1
0,"moved, sad",3,4,3,1,1,1,3,1,1,Sad,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,DISREGARD,9,9,9,5,5,5,5,5,5,Afraid,1,1,1
2,DISREGARD,9,9,9,5,5,5,5,5,5,Afraid,1,1,1
2,DISREGARD,9,9,9,5,5,5,5,5,5,Afraid,1,1,1
2,DISREGARD,9,9,9,5,5,5,5,5,5,Afraid,1,1,1


In [41]:
formal_emotions = emo_labels['Emo']

In [42]:
formal_emotions = formal_emotions.to_numpy()

In [43]:
formal_emotions = formal_emotions.reshape((formal_emotions.size, 1))

In [44]:
from sklearn.preprocessing import OneHotEncoder
cat = OneHotEncoder()
emotions_onehot = cat.fit_transform(formal_emotions).toarray()
emotions_onehot

array([[0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 

In [45]:
net.forward(Variable(torch.Tensor(x_train)))

tensor([[0.4190, 0.5810],
        [0.4225, 0.5775],
        [0.4635, 0.5365],
        [0.3655, 0.6345],
        [0.6712, 0.3288],
        [0.7150, 0.2850],
        [0.5300, 0.4700],
        [0.5441, 0.4559],
        [0.4731, 0.5269],
        [0.7669, 0.2331],
        [0.6581, 0.3419],
        [0.7458, 0.2542],
        [0.4056, 0.5944],
        [0.4640, 0.5360],
        [0.5538, 0.4462],
        [0.6663, 0.3337],
        [0.6237, 0.3763],
        [0.6418, 0.3582],
        [0.3801, 0.6199],
        [0.5881, 0.4119],
        [0.6624, 0.3376],
        [0.4165, 0.5835],
        [0.6837, 0.3163],
        [0.7260, 0.2740],
        [0.6144, 0.3856],
        [0.8387, 0.1613],
        [0.3359, 0.6641],
        [0.2004, 0.7996],
        [0.2442, 0.7558],
        [0.3549, 0.6451],
        [0.3922, 0.6078],
        [0.7283, 0.2717],
        [0.6744, 0.3256],
        [0.5013, 0.4987],
        [0.7975, 0.2025],
        [0.6888, 0.3112],
        [0.4949, 0.5051],
        [0.3820, 0.6180],
        [0.4